# II - Recommendations

## Train recommendation algorithms

In [1]:
from zero.dataset import Dataset

triplets = Rating.objects.values_list('user_id', 'work_id', 'choice')
dataset = Dataset()
anonymized = dataset.make_anonymous_data(triplets)

In [2]:
from zero.als import MangakiALS

algo = MangakiALS()
algo.set_parameters(anonymized.nb_users, anonymized.nb_works)
algo.fit(anonymized.X, anonymized.y)

Computing M: (2220 × 11090)


Chrono: fill and center matrix [1002ms]
Chrono: factor matrix [19370ms]


Shapes (2220, 20) (20, 11090)


## On a couple of works

In [3]:
Work.objects.filter(title__search='Rakugo').values_list('id', 'title')

<WorkQuerySet [(12780, 'Shouwa Genroku Rakugo Shinjuu (2016)'), (9060, 'Rakugo Tennyo Oyui'), (17000, 'Descending Stories: Showa Genroku Rakugo Shinju'), (12778, 'Shouwa Genroku Rakugo Shinjuu: Yotarou Hourou-hen')]>

In [4]:
users = [User.objects.get(username=username) for username in {'jj', 'RaitoBezarius'}]
works = [Work.objects.get(title=title)
         for title in {'Steins;Gate 0', 'Penguin Highway', 'Shouwa Genroku Rakugo Shinjuu (2016)'}]

In [5]:
names = {user.id: user.username for user in users}
titles = {work.id: work.title for work in Work.objects.all()}

In [6]:
import numpy as np

X_test = np.array([[dataset.encode_user[user.id],
                    dataset.encode_work[work.id]] for user in users for work in works])
pred = algo.predict(X_test)

In [7]:
for (enc_user_id, enc_work_id), rating in zip(X_test, pred):
    user_id = dataset.decode_user[enc_user_id]
    work_id = dataset.decode_work[enc_work_id]
    print('{:s} on {:s} should give a rating of {:f}'.format(names[user_id], titles[work_id], rating))

jj on Steins;Gate 0 should give a rating of 1.318260
jj on Penguin Highway should give a rating of 2.883078
jj on Shouwa Genroku Rakugo Shinjuu (2016) should give a rating of 2.188034
RaitoBezarius on Steins;Gate 0 should give a rating of 2.121159
RaitoBezarius on Penguin Highway should give a rating of -0.815448
RaitoBezarius on Shouwa Genroku Rakugo Shinjuu (2016) should give a rating of 0.207930


## On the whole Mangaki database

In [8]:
from itertools import product
import pandas as pd

X_test = list(product(names.keys(), range(dataset.anonymized.nb_works)))

df = pd.DataFrame(X_test, columns=('user', 'enc_work'))

In [9]:
df['enc_user'] = df['user'].map(dataset.encode_user)
df['work'] = df['enc_work'].map(dataset.decode_work)

df['username'] = df['user'].map(names)
df['title'] = df['work'].map(titles)

In [10]:
df.head()

,user,enc_work,enc_user,work,username,title
0,1,0,542,10923,jj,Isuca
1,1,1,542,635,jj,RahXephon Interlude: Her and Herself/Thatness ...
2,1,2,542,780,jj,Ghost in the Shell: Arise - Border:2 Ghost Whi...
3,1,3,542,4504,jj,Big Bang Vénus
4,1,4,542,10349,jj,Minami-ke Okaeri


In [11]:
pred = algo.predict(np.array(df[['enc_user', 'enc_work']]))

In [12]:
df['pred'] = pred

In [13]:
df = df.sort_values('pred', ascending=False)

In [14]:
df.head(10)

,user,enc_work,enc_user,work,username,title,pred
20803,33,9713,936,15547,RaitoBezarius,The Fruit of Grisaia,3.251786
16252,33,5162,936,2098,RaitoBezarius,Grisaia no Meikyuu,3.198894
21886,33,10796,936,20851,RaitoBezarius,Darling in the FranXX,3.127811
18355,33,7265,936,13818,RaitoBezarius,Utawarerumono: The False Faces,3.069175
18517,33,7427,936,14722,RaitoBezarius,The Seven Deadly Sins,3.057923
18808,33,7718,936,13809,RaitoBezarius,Kamisama Hajimemashita: Kako-hen,2.982179
20317,33,9227,936,18314,RaitoBezarius,Made in Abyss,2.964089
6226,1,6226,542,18416,jj,"The Night is Short, Walk On Girl",2.926828
16405,33,5315,936,8273,RaitoBezarius,Inu X Boku Secret Service Special,2.915946
1924,1,1924,542,17542,jj,Restaurant to Another World,2.897161
